## Importando bibliotecas

In [ ]:
# Ocultando mensagens do sistema
import warnings
warnings.filterwarnings('ignore')

# Bibliotecas iniciais
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Inspeção Inicial dos dados

Os dados deste dataset foram preparados neste mesmo repositório do GitHub, em um notebook nomeado `Dataset.jpynb` , onde foram concatenados cerca de cinco datasets com avaliações de restaurantes em diferentes países.

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/CSVs/restaurantes.csv')
df = pd.DataFrame(data)

In [ ]:
print(f"\n↳ Quantidade de linhas: {df.shape[0]}")
print(f"↳ Quantidade de colunas: {df.shape[1]}\n")
df.head(3)


↳ Quantidade de linhas: 2299006
↳ Quantidade de colunas: 13



,Unnamed: 0,parse_count,restaurant_name,rating_review,sample,review_id,title_review,review_preview,review_full,date,city,url_restaurant,author_id
0,0,1,Chalito_Rambla,1,Negative,review_774086112,Terrible food Terrible service,"Ok, this place is terrible! Came here bc we’ve...","Ok, this place is terrible! Came here bc we’ve...","October 12, 2020",Barcelona_Catalonia,https://www.tripadvisor.com/Restaurant_Review-...,UID_0
1,1,2,Chalito_Rambla,5,Positive,review_739142140,The best milanesa in central Barcelona,This place was a great surprise. The food is d...,This place was a great surprise. The food is d...,"January 14, 2020",Barcelona_Catalonia,https://www.tripadvisor.com/Restaurant_Review-...,UID_1
2,2,3,Chalito_Rambla,5,Positive,review_749758638,Family bonding,The food is excellent.....the ambiance is very...,The food is excellent.....the ambiance is very...,"March 7, 2020",Barcelona_Catalonia,https://www.tripadvisor.com/Restaurant_Review-...,UID_2


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2299006 entries, 0 to 2299005
Data columns (total 13 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   Unnamed: 0       object
 1   parse_count      object
 2   restaurant_name  object
 3   rating_review    object
 4   sample           object
 5   review_id        object
 6   title_review     object
 7   review_preview   object
 8   review_full      object
 9   date             object
 10  city             object
 11  url_restaurant   object
 12  author_id        object
dtypes: object(13)
memory usage: 228.0+ MB


## Limpeza, Manipulação e Preparação

Para este projeto, irei precisar apenas de algumas colunas desse dataset, sendo elas:

- `rating_review` é utilizada para analisar a pontuação do usuário, de `1` à `5` (do pior para o melhor).
- `sample` é a coluna que contém uma avaliação de sentimento do comentário, sendo ela `Negative` ou `Positive` .
- `review_full` contém o comentário completo dos usuário em relação ao restaurante, todos eles estão em inglês.

**Objetivos Iniciais**
- Criar um novo DataFrame com as 3 tabelas citadas acima, renomeando-as para se encaixar melhor ao propósito deste projeto.
- Identificar se existe um desbalanceamento na quantidade de avaliações positivas e negativas.
- Estudar o impacto das pontuações de `1` à `5` e estudar possibilidade de classificar um novo sentimento como `Neutral` .


In [54]:
# Preparando novo dataframe com as colunas 'rating_review', 'sample' e 'review_full'
df_pln = df[['rating_review', 'sample', 'review_full']]

# Renomenado colunas
df_pln = df_pln.rename(columns={'rating_review': 'ratings', 'sample': 'sentiments', 'review_full': 'reviews'})

df_pln.head()

,ratings,sentiments,reviews
0,1,Negative,"Ok, this place is terrible! Came here bc we’ve..."
1,5,Positive,This place was a great surprise. The food is d...
2,5,Positive,The food is excellent.....the ambiance is very...
3,5,Positive,"The food is execellent ,affortable price for p..."
4,5,Positive,"Mr Suarez,The food at your restaurant was abso..."


In [55]:
# Porcentagem de valores faltantes
print(f"Quantidade de valores ausentes:\n{df_pln.isna().sum()}")

Quantidade de valores ausentes:
ratings           9
sentiments        9
reviews       51199
dtype: int64


In [56]:
# Observando os valores ausentes
df_pln[df_pln['reviews'].isnull()].head(10)

,ratings,sentiments,reviews
187262,4,Positive,NaN
187263,Barcelona_Catalonia,https://www.tripadvisor.com/Restaurant_Review-...,NaN
612489,4.0,Positive,NaN
612490,London_England,https://www.tripadvisor.com/Restaurant_Review-...,NaN
828375,5.0,Positive,NaN
828376,NaN,NaN,NaN
828377,NaN,NaN,NaN
828378,NaN,NaN,NaN
828379,London_England,https://www.tripadvisor.com/Restaurant_Review-...,NaN
1009105,5.0,Positive,NaN


In [57]:
# Dropando valores ausentes
df_pln = df_pln.dropna()
print(f"Quantidade de valores ausentes:\n{df_pln.isna().sum()}")

Quantidade de valores ausentes:
ratings       0
sentiments    0
reviews       0
dtype: int64


In [58]:
# Contagem de valores categóricos
df_categ = df_pln.groupby(['ratings', 'sentiments']).size().reset_index(name='Count').sort_values(by='Count', ascending=False)
df_categ

,ratings,sentiments,Count
4,5,Positive,967675
3,4,Positive,463400
2,3,Negative,160845
14,5.0,Positive,152569
13,5,Positive,136328
0,1,Negative,75039
1,2,Negative,73495
11,4,Positive,70711
12,4.0,Positive,67050
10,3.0,Negative,22905


In [59]:
# Transformando a coluna 'rating' em float
df_categ['ratings'] = df_categ['ratings'].astype(float)

# Agregando a coluna em rating
df_rating = df_categ.groupby(['ratings', 'sentiments']).agg({'Count': 'sum'}).reset_index()
df_rating.sort_values(by='ratings', ascending=False)

,ratings,sentiments,Count
4,5.0,Positive,1256572
3,4.0,Positive,601161
2,3.0,Negative,205974
1,2.0,Negative,92307
0,1.0,Negative,91793


In [60]:
# Preparando variáveis para plotagem
positive_count = df_rating.loc[df_rating['sentiments'] == 'Positive', 'Count'].values[0]
negative_count = df_rating.loc[df_rating['sentiments'] == 'Negative', 'Count'].values[0]

# Cores para os plots
cores = ['#d62828', '#ff9e00', '#eeef20', '#aacc00', '#55a630']

In [61]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

#------------------
# Primeiro subplot: Gráfico de pizza (intensidade)
#------------------
axes[0].pie([positive_count, negative_count],
            labels=['Positive', 'Negative'],
            colors= [cores[4], cores[0]],
            autopct='%1.1f%%',
            startangle=90,
            explode=(0.1, 0))

axes[0].set_title('Intensidade das Avaliações', fontsize=14, fontweight='bold')
axes[0].axis('equal')

#------------------
# Segundo subplot: Gráfico de barras (contagem)
#------------------
ax = sns.barplot(x=df_rating['ratings'],
                 y=df_rating['Count'],
                 palette=cores, ax=axes[1])

for i in range(len(df_rating['ratings'])):
    axes[1].annotate(f'{df_rating["Count"][i]:.0f}'.replace('.', ','),
                (i, df_rating['Count'][i]),
                ha="center",
                xytext=(0, 3),
                textcoords="offset points",
                fontweight='bold')

# Ocultando eixos e algumas bordas
axes[1].yaxis.set_visible(False)
axes[1].spines["top"].set_visible(False)
axes[1].spines["left"].set_visible(False)
axes[1].spines["right"].set_visible(False)

axes[1].set_title('Contagem por Avaliação', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Nota da Avaliação')
axes[1].set_ylabel('Contagem')

plt.tight_layout()
plt.show()

In [62]:
df_rating = df_rating.reset_index(drop=True)
df_pln = df_pln.reset_index(drop=True)

df_pln = pd.merge(df_rating, df_pln[['ratings', 'sentiments', 'reviews']], on=['ratings', 'sentiments'], how='left')
df_pln['ratings'] = df_pln['ratings'].astype(int)

In [63]:
df_pln.head(3)

,ratings,sentiments,Count,reviews
0,1,Negative,91793,"Ok, this place is terrible! Came here bc we’ve..."
1,1,Negative,91793,I always enjoyed above the average-to-outstand...
2,1,Negative,91793,second visit there and i am very disapointed t...


In [64]:
nota_3 = df_pln[df_pln['ratings'] == 3]
print("➥ Analisando os sentimentos de alguns comentários com nota 3:\n")
for i, review in enumerate(nota_3.iloc[0:5, :]['reviews'], start=1):
    print(f"{i}ª Review: {review}\n")
print("...")

➥ Analisando os sentimentos de alguns comentários com nota 3:

1ª Review: We went there for dinner further a friend’s recommendation. The menu is pretty large, the service was a bit disappointing, when we asked few recommendations to the waiter about the wine for instance, you need to push to have some advises. The food was acceptable, we had a carpaccio of octopus as a starter which was a bit disappointing too (couldn’t feel properly the taste of the octopus). Regarding the pizza they were very yummy.

2ª Review: Um, how this joint has this many 5 star reviews will FOREVER vex me. Dramatic presentation, for sure, but the execution was spotty at best for such an extremely focused menu. Will not return.

3ª Review: I am so confused by the positive reviews of the food. This is just not good Spanish food. Our drinks were great, but the suckling pig was dry and tough. We ate none of it because it was so bad. We had the spring rolls, which were good and a few other items. The server not ver